<a href="https://colab.research.google.com/github/TNTanKhai/TTNT/blob/main/Tuan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import các thư viện cần thiết
import copy    # Để tạo bản sao sâu của danh sách
import math    # Để sử dụng giá trị vô cực trong thuật toán
import random  # Để tạo số ngẫu nhiên (nếu cần)
import numpy   # Để hiển thị bảng cờ dạng ma trận đẹp hơn

# Định nghĩa các ký hiệu cho trò chơi
X = "X"        # Người chơi X
O = "O"        # Người chơi O
EMPTY = None   # Ô trống trên bàn cờ
user = None    # Biến lưu ký hiệu của người dùng (X hoặc O)
ai = None      # Biến lưu ký hiệu của máy (đối thủ của người dùng)
board_size = 3 # Kích thước bàn cờ (mặc định 3x3)

def initial_state():
    """
    Tạo trạng thái ban đầu của bàn cờ.
    Tất cả các ô đều trống.
    """
    # Tạo ma trận board_size x board_size với tất cả giá trị EMPTY
    return [[EMPTY for _ in range(board_size)] for _ in range(board_size)]

def player(board):
    """
    Xác định người chơi có lượt đi tiếp theo.
    """
    count = 0  # Biến đếm số nước đi đã thực hiện

    # Duyệt qua tất cả các ô trên bàn cờ
    for i in board:      # i là một hàng trong bàn cờ
        for j in i:      # j là một ô trong hàng
            if j:        # Nếu ô không trống (có giá trị X hoặc O)
                count += 1  # Tăng biến đếm

    # Logic: X luôn đi trước, nếu số nước đi lẻ thì O đi, chẵn thì X đi
    if count % 2 != 0:   # Nếu số nước đi là số lẻ
        return ai        # Trả về AI (vì đã đến lượt của đối thủ)
    return user          # Ngược lại, trả về User

def actions(board):
    """
    Trả về tập hợp tất cả các nước đi có thể thực hiện.
    """
    res = set()  # Tạo một tập hợp rỗng để lưu các nước đi hợp lệ

    # Duyệt qua tất cả các ô trên bàn cờ
    for i in range(board_size):      # Duyệt theo hàng
        for j in range(board_size):  # Duyệt theo cột
            if board[i][j] == EMPTY: # Nếu ô hiện tại trống
                res.add((i, j))      # Thêm vị trí (i,j) vào tập hợp kết quả

    return res  # Trả về tập hợp các nước đi có thể

def result(board, action):
    """
    Tạo bàn cờ mới sau khi thực hiện nước đi.
    """
    # Xác định người chơi hiện tại
    curr_player = player(board)

    # Tạo bản sao sâu của bàn cờ hiện tại để không làm thay đổi bàn cờ gốc
    result_board = copy.deepcopy(board)

    # Lấy tọa độ từ action
    (i, j) = action

    # Đặt ký hiệu của người chơi hiện tại vào vị trí (i,j)
    result_board[i][j] = curr_player

    return result_board  # Trả về bàn cờ mới

def get_horizontal_winner(board):
    """
    Kiểm tra người thắng theo hàng ngang.
    """
    winner_val = None  # Biến lưu người thắng, ban đầu là None

    # Duyệt qua từng hàng
    for i in range(board_size):
        # Giả sử người thắng là người ở ô đầu tiên của hàng
        winner_val = board[i][0]

        # Kiểm tra tất cả các ô trong hàng có giống ô đầu tiên không
        for j in range(board_size):
            if board[i][j] != winner_val:  # Nếu có ô khác giá trị
                winner_val = None           # Đặt lại thành None
                break                       # Thoát vòng lặp sớm

        # Nếu tìm thấy người thắng, trả về kết quả ngay
        if winner_val:
            return winner_val

    return winner_val  # Trả về None nếu không có người thắng theo hàng

def get_vertical_winner(board):
    """
    Kiểm tra người thắng theo hàng dọc.
    """
    winner_val = None  # Biến lưu người thắng, ban đầu là None

    # Duyệt qua từng cột
    for i in range(board_size):
        # Giả sử người thắng là người ở ô đầu tiên của cột
        winner_val = board[0][i]

        # Kiểm tra tất cả các ô trong cột có giống ô đầu tiên không
        for j in range(board_size):
            if board[j][i] != winner_val:  # Nếu có ô khác giá trị
                winner_val = None           # Đặt lại thành None
                break                       # Thoát vòng lặp sớm

        # Nếu tìm thấy người thắng, trả về kết quả ngay
        if winner_val:
            return winner_val

    return winner_val  # Trả về None nếu không có người thắng theo cột

def get_diagonal_winner(board):
    """
    Kiểm tra người thắng theo đường chéo.
    """
    winner_val = None  # Biến lưu người thắng

    # Kiểm tra đường chéo chính (từ trái trên sang phải dưới)
    winner_val = board[0][0]  # Ô đầu tiên của đường chéo
    for i in range(board_size):
        if board[i][i] != winner_val:  # Nếu có ô khác giá trị
            winner_val = None           # Đặt lại thành None
            break                       # Thoát vòng lặp

    # Nếu tìm thấy người thắng, trả về kết quả
    if winner_val:
        return winner_val

    # Kiểm tra đường chéo phụ (từ phải trên sang trái dưới)
    winner_val = board[0][board_size - 1]  # Ô đầu tiên của đường chéo phụ
    for i in range(board_size):
        j = board_size - 1 - i  # Tính vị trí cột cho đường chéo phụ
        if board[i][j] != winner_val:  # Nếu có ô khác giá trị
            winner_val = None           # Đặt lại thành None
            break                       # Thoát vòng lặp

    return winner_val  # Trả về kết quả (có thể là None)

def winner(board):
    """
    Xác định người thắng cuộc của trò chơi.
    """
    # Kiểm tra lần lượt: hàng ngang, hàng dọc, đường chéo
    # Nếu không có ai thắng thì trả về None
    winner_val = get_horizontal_winner(board) or get_vertical_winner(board) or get_diagonal_winner(board) or None
    return winner_val

def terminal(board):
    """
    Kiểm tra trò chơi đã kết thúc chưa.
    """
    # Nếu đã có người thắng, trò chơi kết thúc
    if winner(board) != None:
        return True

    # Kiểm tra nếu còn ô trống nào không
    for i in board:      # Duyệt qua từng hàng
        for j in i:      # Duyệt qua từng ô trong hàng
            if j == EMPTY:  # Nếu tìm thấy ô trống
                return False  # Trò chơi chưa kết thúc

    # Nếu không còn ô trống nào, trò chơi kết thúc (hòa)
    return True

def utility(board):
    """
    Tính điểm utility của bàn cờ.
    """
    winner_val = winner(board)  # Xác định người thắng

    if winner_val == X:  # Nếu X thắng
        return 1         # Trả về 1
    elif winner_val == O:  # Nếu O thắng
        return -1        # Trả về -1
    return 0             # Nếu hòa, trả về 0

def maxValue(state):
    """
    Hàm MAX trong thuật toán Minimax - tìm nước đi tốt nhất cho MAX (X).
    """
    # Nếu đây là trạng thái kết thúc, trả về utility
    if terminal(state):
        return utility(state)

    v = -math.inf  # Khởi tạo giá trị tốt nhất là âm vô cực

    # Duyệt qua tất cả các nước đi có thể
    for action in actions(state):
        # Gọi đệ quy hàm MIN và cập nhật giá trị tốt nhất
        v = max(v, minValue(result(state, action)))

    return v  # Trả về giá trị tốt nhất tìm được

def minValue(state):
    """
    Hàm MIN trong thuật toán Minimax - tìm nước đi tốt nhất cho MIN (O).
    """
    # Nếu đây là trạng thái kết thúc, trả về utility
    if terminal(state):
        return utility(state)

    v = math.inf  # Khởi tạo giá trị tốt nhất là dương vô cực

    # Duyệt qua tất cả các nước đi có thể
    for action in actions(state):
        # Gọi đệ quy hàm MAX và cập nhật giá trị tốt nhất
        v = min(v, maxValue(result(state, action)))

    return v  # Trả về giá trị tốt nhất tìm được

def minimax(board):
    """
    Thuật toán Minimax - tìm nước đi tối ưu cho người chơi hiện tại.
    """
    current_player = player(board)  # Xác định người chơi hiện tại

    if current_player == X:  # Nếu là lượt của X (MAX player)
        min_val = -math.inf  # Khởi tạo giá trị tốt nhất là âm vô cực

        # Duyệt qua tất cả các nước đi có thể
        for action in actions(board):
            # Tính giá trị của nước đi này bằng cách gọi hàm MIN
            check = minValue(result(board, action))

            # Nếu nước đi này tốt hơn, cập nhật
            if check > min_val:
                min_val = check    # Cập nhật giá trị tốt nhất
                move = action      # Lưu nước đi tốt nhất

    else:  # Nếu là lượt của O (MIN player)
        max_val = math.inf  # Khởi tạo giá trị tốt nhất là dương vô cực

        # Duyệt qua tất cả các nước đi có thể
        for action in actions(board):
            # Tính giá trị của nước đi này bằng cách gọi hàm MAX
            check = maxValue(result(board, action))

            # Nếu nước đi này tốt hơn, cập nhật
            if check < max_val:
                max_val = check    # Cập nhật giá trị tốt nhất
                move = action      # Lưu nước đi tốt nhất

    return move  # Trả về nước đi tối ưu

if __name__ == "__main__":
    """
    Hàm main - khởi chạy trò chơi.
    """
    # Cho phép người dùng chọn kích thước bàn cờ
    while True:
        try:
            size_input = input("Enter board size (3 for 3x3, 4 for 4x4, etc.): ")
            board_size = int(size_input)  # Chuyển đổi input thành số
            if board_size >= 3:  # Kiểm tra kích thước hợp lệ
                break
            else:
                print("Size must be at least 3.")
        except ValueError:  # Bắt lỗi nếu input không phải số
            print("Please enter a valid number.")

    # Khởi tạo bàn cờ với kích thước đã chọn
    board = initial_state()
    ai_turn = False  # Biến xác định có phải lượt của AI không

    # Cho người dùng chọn ký hiệu (X hoặc O)
    print("Choose a player")
    user = input()  # Nhập ký hiệu từ người dùng

    # Xác định ký hiệu của AI dựa trên lựa chọn của người dùng
    if user == "X":
        ai = "O"  # Nếu user chọn X thì AI là O
    else:
        ai = "X"  # Nếu user chọn O thì AI là X

    # Vòng lặp chính của trò chơi
    while True:
        game_over = terminal(board)  # Kiểm tra trò chơi đã kết thúc chưa
        playr = player(board)        # Xác định người chơi hiện tại

        if game_over:  # Nếu trò chơi đã kết thúc
            winner_val = winner(board)  # Xác định người thắng

            if winner_val is None:  # Nếu không có người thắng (hòa)
                print("Game Over: Tie.")
            else:  # Nếu có người thắng
                print(f"Game Over: {winner_val} wins.")
            break  # Thoát khỏi vòng lặp, kết thúc trò chơi

        else:  # Nếu trò chơi chưa kết thúc
            if user != playr and not game_over:  # Nếu đến lượt AI
                if ai_turn:  # Nếu đã được đánh dấu là lượt của AI
                    move = minimax(board)        # AI tính nước đi tối ưu
                    board = result(board, move)  # Thực hiện nước đi
                    ai_turn = False              # Đánh dấu đã đi xong
                    print(numpy.array(board))    # Hiển thị bàn cờ

            elif user == playr and not game_over:  # Nếu đến lượt người dùng
                ai_turn = True  # Đánh dấu lượt tiếp theo là của AI
                print("Enter the position to move (row,col)")

                # Nhập tọa độ từ người dùng
                i = int(input("Row:"))  # Nhập số hàng
                j = int(input("Col:"))  # Nhập số cột

                # Kiểm tra nước đi hợp lệ và thực hiện
                if board[i][j] == EMPTY:  # Nếu ô trống
                    board = result(board, (i, j))  # Thực hiện nước đi
                    print(numpy.array(board))      # Hiển thị bàn cờ

In [ ]:
# Import các thư viện cần thiết
import os      # Để xóa màn hình console
import math    # Để sử dụng giá trị vô cực trong thuật toán Minimax

# Khai báo biến toàn cục
BOARD_SIZE = 3   # Kích thước bàn cờ (mặc định 3x3)
WIN_LENGTH = 3   # Số ô liên tiếp cần để thắng

def GetWinner(board):
    """
    Kiểm tra và trả về người thắng cuộc trên bàn cờ.
    """
    size = BOARD_SIZE  # Lấy kích thước bàn cờ

    # Kiểm tra các hàng ngang - duyệt qua từng hàng
    for i in range(size):  # i là chỉ số hàng (0 → size-1)
        # Duyệt qua các vị trí bắt đầu có thể tạo thành chuỗi WIN_LENGTH
        for j in range(size - WIN_LENGTH + 1):
            # Kiểm tra WIN_LENGTH ô liên tiếp trong hàng i
            # all() trả về True nếu tất cả các ô đều giống nhau
            if all(board[i*size + j + k] == board[i*size + j] for k in range(WIN_LENGTH)) and board[i*size + j] in ["X", "O"]:
                return board[i*size + j]  # Trả về "X" hoặc "O" nếu thắng

    # Kiểm tra các cột dọc - duyệt qua từng cột
    for i in range(size - WIN_LENGTH + 1):  # Vị trí bắt đầu theo hàng
        for j in range(size):               # j là chỉ số cột
            # Kiểm tra WIN_LENGTH ô liên tiếp trong cột j
            if all(board[(i+k)*size + j] == board[i*size + j] for k in range(WIN_LENGTH)) and board[i*size + j] in ["X", "O"]:
                return board[i*size + j]  # Trả về "X" hoặc "O" nếu thắng

    # Kiểm tra đường chéo chính (\) - từ trái trên sang phải dưới
    for i in range(size - WIN_LENGTH + 1):      # Vị trí bắt đầu hàng
        for j in range(size - WIN_LENGTH + 1):  # Vị trí bắt đầu cột
            # Kiểm tra đường chéo chính
            if all(board[(i+k)*size + (j+k)] == board[i*size + j] for k in range(WIN_LENGTH)) and board[i*size + j] in ["X", "O"]:
                return board[i*size + j]  # Trả về "X" hoặc "O" nếu thắng

    # Kiểm tra đường chéo phụ (/) - từ phải trên sang trái dưới
    for i in range(size - WIN_LENGTH + 1):          # Vị trí bắt đầu hàng
        for j in range(WIN_LENGTH - 1, size):       # Vị trí bắt đầu cột (từ cuối)
            # Kiểm tra đường chéo phụ
            if all(board[(i+k)*size + (j-k)] == board[i*size + j] for k in range(WIN_LENGTH)) and board[i*size + j] in ["X", "O"]:
                return board[i*size + j]  # Trả về "X" hoặc "O" nếu thắng

    return None  # Không có người thắng

def PrintBoard(board):
    """
    Xóa màn hình và in bàn cờ hiện tại.
    """
    # Xóa màn hình console: 'cls' cho Windows, 'clear' cho Linux/Mac
    os.system('cls' if os.name=='nt' else 'clear')
    size = BOARD_SIZE  # Lấy kích thước bàn cờ

    # In tiêu đề với thông tin kích thước và điều kiện thắng
    print(f"\nBoard {size}x{size} (Win with {WIN_LENGTH} in a row)\n")

    # Tính chiều rộng tối đa cho mỗi ô (để căn chỉnh đẹp)
    max_width = len(str(size * size))  # Độ dài của số lớn nhất trên bàn cờ

    # In từng hàng của bàn cờ
    for i in range(size):  # Duyệt qua từng hàng
        row = []  # Danh sách chứa các ô trong hàng hiện tại
        for j in range(size):  # Duyệt qua từng cột trong hàng
            cell = board[i*size + j]  # Lấy giá trị ô hiện tại

            if cell in ["X", "O"]:  # Nếu ô có "X" hoặc "O"
                # Căn giữa ký tự X/O
                row.append(f"{cell:^{max_width}}")
            else:  # Nếu ô là số (chưa được đánh)
                # Căn phải số
                row.append(f"{cell:>{max_width}}")

        # Nối các ô bằng dấu " | " và in ra
        print(" | ".join(row))

        # In đường kẻ ngang giữa các hàng (trừ hàng cuối)
        if i < size - 1:
            print("-" * (size * (max_width + 3) - 3))  # Tính độ dài đường kẻ
    print()  # In dòng trống

def GetAvailableCells(board):
    """
    Trả về danh sách các ô còn trống trên bàn cờ.
    """
    available = list()  # Tạo danh sách rỗng để lưu các ô trống

    # Duyệt qua tất cả các ô trên bàn cờ
    for cell in board:
        # Nếu ô không phải "X" hoặc "O" (tức là còn trống)
        if cell != "X" and cell != "O":
            available.append(cell)  # Thêm số ô vào danh sách

    return available  # Trả về danh sách các ô trống

def minimax(position, depth, alpha, beta, isMaximizing):
    """
    Thuật toán Minimax với cắt tỉa Alpha-Beta.
    Trả về giá trị tốt nhất của một nước đi.
    """
    # Kiểm tra nếu đã có người thắng
    winner = GetWinner(position)
    if winner != None:  # Nếu có người thắng
        # Trả về điểm số: ưu tiên thắng nhanh (100 - depth)
        return (100 - depth) if winner == "X" else (-100 + depth)

    # Kiểm tra nếu bàn cờ đã đầy (hòa)
    if len(GetAvailableCells(position)) == 0:
        return 0  # Điểm hòa

    # Nếu là người chơi tối đa (MAX - X)
    if isMaximizing:
        maxEval = -math.inf  # Khởi tạo giá trị tốt nhất là âm vô cực

        # Duyệt qua tất cả các nước đi có thể
        for cell in GetAvailableCells(position):
            # Thử nước đi
            position[cell - 1] = "X"
            # Gọi đệ quy cho người chơi MIN
            Eval = minimax(position, depth + 1, alpha, beta, False)
            # Cập nhật giá trị tốt nhất
            maxEval = max(maxEval, Eval)
            # Cập nhật alpha
            alpha = max(alpha, Eval)
            # Hoàn tác nước đi thử
            position[cell - 1] = cell
            # Cắt tỉa Alpha-Beta
            if beta <= alpha:
                break  # Dừng tìm kiếm ở nhánh này
        return maxEval  # Trả về giá trị tốt nhất tìm được

    else:  # Nếu là người chơi tối thiểu (MIN - O)
        minEval = +math.inf  # Khởi tạo giá trị tốt nhất là dương vô cực

        # Duyệt qua tất cả các nước đi có thể
        for cell in GetAvailableCells(position):
            # Thử nước đi
            position[cell - 1] = "O"
            # Gọi đệ quy cho người chơi MAX
            Eval = minimax(position, depth + 1, alpha, beta, True)
            # Cập nhật giá trị tốt nhất
            minEval = min(minEval, Eval)
            # Cập nhật beta
            beta = min(beta, Eval)
            # Hoàn tác nước đi thử
            position[cell - 1] = cell
            # Cắt tỉa Alpha-Beta
            if beta <= alpha:
                break  # Dừng tìm kiếm ở nhánh này
        return minEval  # Trả về giá trị tốt nhất tìm được

def FindBestMove(currentPosition, AI):
    """
    Tìm nước đi tốt nhất cho AI.
    """
    # Khởi tạo giá trị tốt nhất tùy thuộc vào AI là X hay O
    bestVal = -math.inf if AI == "X" else +math.inf
    bestMove = -1  # Khởi tạo nước đi tốt nhất

    # Lấy danh sách các nước đi có thể
    available_moves = GetAvailableCells(currentPosition)

    # Tối ưu hóa cho bàn cờ lớn: sử dụng chiến thuật đơn giản nếu có quá nhiều nước đi
    if BOARD_SIZE > 3 and len(available_moves) > 9:
        # Tính ô trung tâm
        center = (BOARD_SIZE * BOARD_SIZE + 1) // 2
        # Tính 4 góc
        corners = [1, BOARD_SIZE, BOARD_SIZE * (BOARD_SIZE - 1) + 1, BOARD_SIZE * BOARD_SIZE]

        # Ưu tiên ô trung tâm
        for move in available_moves:
            if move == center:
                return move  # Trả về ô trung tâm ngay lập tức
        # Ưu tiên các góc
        for move in available_moves:
            if move in corners:
                return move  # Trả về một trong các góc
        return available_moves[0]  # Trả về nước đi đầu tiên nếu không có ưu tiên

    # Duyệt qua tất cả các nước đi có thể
    for cell in available_moves:
        # Thử nước đi
        currentPosition[cell - 1] = AI
        # Tính giá trị của nước đi này bằng Minimax
        moveVal = minimax(currentPosition, 0, -math.inf, +math.inf, False if AI == "X" else True)
        # Hoàn tác nước đi thử
        currentPosition[cell - 1] = cell

        # Cập nhật nước đi tốt nhất
        if (AI == "X" and moveVal > bestVal):  # Nếu AI là X (MAX)
            bestMove = cell
            bestVal = moveVal
        elif (AI == "O" and moveVal < bestVal):  # Nếu AI là O (MIN)
            bestMove = cell
            bestVal = moveVal

    return bestMove  # Trả về nước đi tốt nhất

def initialize_board():
    """
    Khởi tạo bàn cờ dựa trên BOARD_SIZE hiện tại.
    """
    # Tạo danh sách từ 1 đến BOARD_SIZE * BOARD_SIZE
    return [i for i in range(1, BOARD_SIZE * BOARD_SIZE + 1)]

def main():
    """
    Hàm chính - điều khiển luồng trò chơi.
    """
    global BOARD_SIZE, WIN_LENGTH  # Khai báo sử dụng biến toàn cục

    # Nhập kích thước bàn cờ từ người dùng
    while True:
        try:
            size_input = input("Enter board size (e.g., 3 for 3x3, 4 for 4x4): ").strip()
            BOARD_SIZE = int(size_input)  # Chuyển đổi input thành số
            if BOARD_SIZE < 3:  # Kiểm tra kích thước tối thiểu
                print("Board size must be at least 3x3")
                continue

            # Nhập số ô cần để thắng
            win_input = input(f"Enter number in a row to win (default {BOARD_SIZE}): ").strip()
            WIN_LENGTH = int(win_input) if win_input else BOARD_SIZE  # Mặc định bằng BOARD_SIZE

            # Kiểm tra điều kiện thắng hợp lệ
            if WIN_LENGTH > BOARD_SIZE:
                print(f"Win length cannot be larger than board size ({BOARD_SIZE})")
                WIN_LENGTH = BOARD_SIZE  # Tự động sửa thành BOARD_SIZE
            elif WIN_LENGTH < 3:
                print("Win length must be at least 3")
                WIN_LENGTH = 3  # Tự động sửa thành 3

            break  # Thoát vòng lặp nếu input hợp lệ
        except ValueError:  # Bắt lỗi nếu input không phải số
            print("Please enter a valid number")

    # Chọn người chơi (X hoặc O)
    player = input("Play as X or O? ").strip().upper()
    while player not in ["X", "O"]:  # Kiểm tra input hợp lệ
        player = input("Please enter X or O: ").strip().upper()

    # Xác định AI là đối thủ của người chơi
    AI = "O" if player == "X" else "X"
    # Khởi tạo bàn cờ
    currentGame = initialize_board()

    # Khởi tạo biến trò chơi
    currentTurn = "X"  # X luôn đi trước
    counter = 0  # Đếm số nước đi
    max_moves = BOARD_SIZE * BOARD_SIZE  # Số nước đi tối đa

    # In thông tin bắt đầu trò chơi
    print(f"\nStarting {BOARD_SIZE}x{BOARD_SIZE} game! Win by getting {WIN_LENGTH} in a row.")
    print("Cells are numbered from top-left to bottom-right:")
    PrintBoard(currentGame)  # Hiển thị bàn cờ ban đầu

    # Vòng lặp chính của trò chơi
    while True:
        # Lượt của AI
        if currentTurn == AI:
            print(f"\nAI ({AI}) is thinking...")
            # AI tìm nước đi tốt nhất
            cell = FindBestMove(currentGame, AI)
            # Thực hiện nước đi
            currentGame[cell - 1] = AI
            # Chuyển lượt cho người chơi
            currentTurn = player
            # Hiển thị bàn cờ sau nước đi của AI
            PrintBoard(currentGame)

        # Lượt của người chơi
        elif currentTurn == player:
            PrintBoard(currentGame)  # Hiển thị bàn cờ
            while True:  # Vòng lặp cho đến khi nhập hợp lệ
                try:
                    # Nhập số ô từ người chơi
                    humanInput = int(input("Enter Cell Number: ").strip())
                    # Kiểm tra ô có hợp lệ và còn trống không
                    if humanInput in currentGame:
                        # Thực hiện nước đi
                        currentGame[humanInput - 1] = player
                        # Chuyển lượt cho AI
                        currentTurn = AI
                        break  # Thoát vòng lặp nhập
                    else:
                        print("Cell Not Available or Invalid.")
                except ValueError:  # Bắt lỗi nhập không phải số
                    print("Please enter a valid number.")

        # Kiểm tra người thắng sau mỗi nước đi
        winner = GetWinner(currentGame)
        if winner != None:  # Nếu có người thắng
            PrintBoard(currentGame)  # Hiển thị bàn cờ cuối cùng
            print(f"{winner} WON!!!")  # Thông báo người thắng
            break  # Kết thúc trò chơi

        # Tăng biến đếm nước đi
        counter += 1
        # Kiểm tra hòa (đã đi hết các ô mà không có người thắng)
        if winner == None and counter == max_moves:
            PrintBoard(currentGame)  # Hiển thị bàn cờ cuối cùng
            print("Tie.")  # Thông báo hòa
            break  # Kết thúc trò chơi

# Chạy chương trình khi được thực thi trực tiếp
if __name__ == "__main__":
    main()